### Age-structured SIR and SEIR model for COVID-19 epidemic in India

In this example, we fit the data to obtain the parameter beta

In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../..')
%run setup.py install
os.chdir(owd)

In [2]:
%matplotlib inline
import numpy as np
import pyross
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import optimize

In [3]:
M=16  # number of age groups

# load age structure data
my_data = np.genfromtxt('../data/age_structures/India-2019.csv', delimiter=',', skip_header=1)
aM, aF = my_data[:, 1], my_data[:, 2]

# set age groups
Ni=aM+aF;   Ni=Ni[0:M];  N=np.sum(Ni)

In [4]:
# contact matrices
CH, CW, CS, CO = pyross.contactMatrix.India()


# matrix of total contacts
C=CH+CW+CS+CO

## SIR

In [5]:
gIa   = 1./7            # recovery rate of asymptomatic infectives 
gIs   = 1./7            # recovery rate of symptomatic infectives 
alpha = 0.               # fraction of asymptomatic infectives 
fsa   = 1                # the self-isolation parameter   


# initial conditions    
Is_0 = np.zeros((M));  Is_0[6:13]=3;  Is_0[2:6]=1
Ia_0 = np.zeros((M))
R_0  = np.zeros((M))
S_0  = Ni - (Ia_0 + Is_0 + R_0)

my_data = np.genfromtxt('../data/covid-cases/india.txt', delimiter='', skip_header=7)
day, cases = my_data[:,0], my_data[:,3]-my_data[:,1]

if alpha !=0:
    cases_mf = 1./(1-alpha)
else:
    cases_mf = 1
cases = cases*cases_mf


# duration of simulation and data file
Tf=21;  Nf=np.size(cases);  

# the contact structure is independent of time 
def contactMatrix(t):
    return C




def findBetaIs(x):    
    parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'fsa':fsa}
    model = pyross.deterministic.SIR(parameters, M, Ni)

    data=model.simulate(S_0, Ia_0, Is_0, contactMatrix, Tf, Nf)
    
    Is = (model.Is(data))
    summedAgesIs = Is.sum(axis=1)
    
    error = np.sum(( cases*(1+alpha)-summedAgesIs)**2)
    return error
    
    
    
    
def findBetaIsandIa(x):    
    parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'fsa':fsa}
    model = pyross.deterministic.SIR(parameters, M, Ni)

    data=model.simulate(S_0, Ia_0, Is_0, contactMatrix, Tf, Nf)
    
    Is = (model.Is(data))
    summedAgesIs = Is.sum(axis=1)
    
    error = np.sum(( cases*(1+alpha)-summedAgesIs)**2)
    return error

In [6]:
x=.1
parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'fsa':fsa}
model = pyross.deterministic.SIR(parameters, M, Ni)

data=model.simulate(S_0, Ia_0, Is_0, contactMatrix, Tf, Nf)

Is = (model.Is(data))
#Is.shape()
#summedAgesIs = Is.sum(axis=1)


In [7]:
beta0  = 0.0

# fit only Is
sol1 = optimize.root(findBetaIs,beta0) 
print('Is only best fit: ', sol1.x)

# fit both Ia and Is
sol2 = optimize.root(findBetaIsandIa,beta0)
print('Is + Ia best fit: ', sol2.x)

Is only best fit:  [0.01646687]
Is + Ia best fit:  [0.01646687]


## SEIR

In [8]:
gIa   = 1./7            # recovery rate of asymptomatic infectives 
gIs   = 1./7            # recovery rate of symptomatic infectives 
alpha = 0.               # fraction of asymptomatic infectives 
gE    = 1./4
fsa   = 1                # the self-isolation parameter   

# initial conditions    
Is_0 = np.zeros((M));  Is_0[6:13]=3;  Is_0[2:6]=1

E_0 = np.zeros((M))
Ia_0 = np.zeros((M))
R_0  = np.zeros((M))
S_0  = Ni - (Ia_0 + Is_0 + R_0)

my_data = np.genfromtxt('../data/covid-cases/india.txt', delimiter='', skip_header=7)
day, cases = my_data[:,0], my_data[:,3]-my_data[:,1]

if alpha !=0:
    cases_mf = 1./(1-alpha)
else:
    cases_mf = 1
cases = cases*cases_mf


# duration of simulation and data file
Tf=21;  Nf=np.size(cases);  

# the contact structure is independent of time 
def contactMatrix(t):
    return C




def findBetaIs(x):
        parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'gE':gE,'fsa':fsa}
        model = pyross.deterministic.SEIR(parameters, M, Ni)
        data=model.simulate(S_0, E_0, Ia_0, Is_0, contactMatrix, Tf, Nf)

        t = data['t']; IC  = np.zeros((Nf))
        for i in range(M):
            IC += data['X'][:,3*M+i] 

        error = np.sum(( cases-IC)**2)
        return error



    
def findBetaIsandIa(x):
        parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'gE':gE,'fsa':fsa}
        model = pyross.deterministic.SEIR(parameters, M, Ni)
        data=model.simulate(S_0, E_0, Ia_0, Is_0, contactMatrix, Tf, Nf)

        t = data['t']; IC  = np.zeros((Nf))
        for i in range(2*M):
            IC += data['X'][:,2*M+i] 

        error = np.sum(( cases-IC)**2)
        return error

In [9]:
beta0  = 0.0

# fit only Is
sol1 = optimize.root(findBetaIs,beta0) 
print('Is only best fit: ', sol1.x)

# fit both Ia and Is
sol2 = optimize.root(findBetaIsandIa,beta0)
print('Is + Ia best fit: ', sol2.x)

Is only best fit:  [0.03258979]
Is + Ia best fit:  [0.03258979]


In [ ]:
gIa   = 1./7            # recovery rate of asymptomatic infectives 
gIs   = 1./7            # recovery rate of symptomatic infectives 
alpha = 0.               # fraction of asymptomatic infectives 
gE    = 1./4
fsa   = 1                # the self-isolation parameter   

# initial conditions    
Is_0 = np.zeros((M));  Is_0[6:13]=3;  Is_0[2:6]=1

E_0 = np.zeros((M))
Ia_0 = np.zeros((M))
R_0  = np.zeros((M))
S_0  = Ni - (Ia_0 + Is_0 + R_0)

my_data = np.genfromtxt('../data/covid-cases/denmark.txt', delimiter='', skip_header=7)
day, cases = my_data[:,0], my_data[:,3]-my_data[:,1]

if alpha !=0:
    cases_mf = 1./(1-alpha)
else:
    cases_mf = 1
cases = cases*cases_mf


# duration of simulation and data file
Tf=21;  Nf=np.size(cases);  

# the contact structure is independent of time 
def contactMatrix(t):
    return C




def findBetaIs(x):
        parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'gE':gE,'fsa':fsa}
        model = pyross.deterministic.SEIR(parameters, M, Ni)
        data=model.simulate(S_0, E_0, Ia_0, Is_0, contactMatrix, Tf, Nf)

        t = data['t']; IC  = np.zeros((Nf))
        for i in range(M):
            IC += data['X'][:,3*M+i] 

        error = np.sum(( cases-IC)**2)
        return error



    
def findBetaIsandIa(x):
        parameters = {'alpha':alpha,'beta':x, 'gIa':gIa,'gIs':gIs,'gE':gE,'fsa':fsa}
        model = pyross.deterministic.SEIR(parameters, M, Ni)
        data=model.simulate(S_0, E_0, Ia_0, Is_0, contactMatrix, Tf, Nf)

        t = data['t']; IC  = np.zeros((Nf))
        for i in range(2*M):
            IC += data['X'][:,2*M+i] 

        error = np.sum(( cases-IC)**2)
        return error